In [10]:
# %pip install deepface
# %pip install opencv-python
# %pip install opencv-python-headless
# %pip install matplotlib
# %pip install tensorflow
# %pip install keras
# %pip install numpy
# %pip install mysql-connector-python
# %pip install pandas
# %pip install scikit-learn
# %pip install scikit-image

In [11]:
import cv2
from deepface import DeepFace
import mysql.connector
import os
from datetime import datetime
from IPython.display import display, Image, clear_output
import time

In [12]:
# Connect to MySQL (XAMPP must be running)
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",  # Replace with MySQL password if set
    database="emotion_detection"
)
cursor = db.cursor()
print("✅ Connected to MySQL Database")


✅ Connected to MySQL Database


In [13]:
# 🔧 Helper Functions

def get_user_by_face_id(face_id):
    cursor.execute("SELECT * FROM users WHERE face_id = %s", (face_id,))
    return cursor.fetchone()

def insert_user(face_id, image_path):
    cursor.execute("INSERT INTO users (face_id, image_path) VALUES (%s, %s)", (face_id, image_path))
    db.commit()
    return cursor.lastrowid

def log_visit(user_id, emotion):
    cursor.execute("INSERT INTO visits (user_id, emotion) VALUES (%s, %s)", (user_id, emotion))
    db.commit()

def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)




In [14]:
# 🎥 Start Webcam
cap = cv2.VideoCapture(0)
ensure_dir("captured_faces")
print("🎬 Webcam Started - Press 'q' in video window to stop")


🎬 Webcam Started - Press 'q' in video window to stop


In [15]:
# Function to format and display DeepFace results
def display_deepface_results(results):
    for result in results:
        print("DeepFace Results:")
        print(f"Dominant Emotion: {result.get('dominant_emotion', 'Unknown')}")
        print(f"Face Confidence: {result.get('face_confidence', 'N/A')}")
        print("Region:")
        region = result.get('region', {})
        for key, value in region.items():
            print(f"  {key}: {value}")
        print("Emotions:")
        emotions = result.get('emotion', {})
        for emotion, confidence in emotions.items():
            print(f"  {emotion}: {confidence:.2f}")
        print()  # Add a blank line for readability

In [16]:
# 🔁 Updated Loop: Recognize or Register by Comparing Faces
known_faces_dir = "known_faces"
ensure_dir(known_faces_dir)

while True:
    ret, frame = cap.read()
    if not ret:
        print("🚫 Failed to capture frame from webcam.")
        break

    try:
        # Analyze the frame for emotions
        results = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        display_deepface_results(results)  # Call the function to display results

        # Extract dominant emotion and region
        dominant_emotion = results[0].get('dominant_emotion', 'Unknown')
        region = results[0].get('region', {})

        # Crop the face safely
        x, y, w, h = region.get('x', 0), region.get('y', 0), region.get('w', 0), region.get('h', 0)
        if w > 0 and h > 0:  # Ensure valid dimensions
            face_img = frame[y:y+h, x:x+w]
        else:
            print("🚫 No face detected in the frame.")
            continue

        matched_id = None

        # Search in known faces
        for file in os.listdir(known_faces_dir):
            known_face_path = os.path.join(known_faces_dir, file)
            try:
                verified = DeepFace.verify(face_img, known_face_path, enforce_detection=False)
                if verified.get("verified"):
                    matched_id = file.split("_")[0]
                    break
            except Exception as e:
                print(f"⚠️ Error verifying face with {file}: {e}")
                continue

        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Save timestamp in a readable format

        if matched_id:
            # Existing user
            user_id = int(matched_id)
            print(f"✅ Recognized User ID = {user_id}")
        else:
            # New user
            user_face_id = str(int(datetime.now().timestamp()))
            face_filename = f"{user_face_id}_{timestamp.replace(':', '-')}.jpg"
            face_path = os.path.join(known_faces_dir, face_filename)
            cv2.imwrite(face_path, face_img)
            user_id = insert_user(user_face_id, face_path)
            print(f"🆕 New user added: ID = {user_id}")

        # Save visit snapshot and log
        full_frame_path = f"captured_faces/user_{user_id}_{timestamp.replace(':', '-')}.jpg"
        cv2.imwrite(full_frame_path, frame)
        log_visit(user_id, dominant_emotion)

        # Save additional details to the database
        try:
            cursor.execute(
                "INSERT INTO visit_details (user_id, timestamp, emotion, image_path) VALUES (%s, %s, %s, %s)",
                (user_id, timestamp, dominant_emotion, full_frame_path)
            )
            db.commit()
        except Exception as e:
            print(f"❌ Database error: {e}")

        # Display info on the camera feed
        cv2.putText(frame, f"User ID: {user_id}", (50, 50),  # Display User ID
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Emotion: {dominant_emotion}", (50, 100),  # Display Dominant Emotion
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    except Exception as e:
        print("🚫 Face not detected or error:", e)

    cv2.imshow("Face Recognition & Emotion Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("📷 Webcam Released & Window Closed")

DeepFace Results:
Dominant Emotion: neutral
Face Confidence: 0.95
Region:
  x: 328
  y: 0
  w: 181
  h: 181
  left_eye: (451, 69)
  right_eye: (375, 65)
Emotions:
  angry: 0.18
  disgust: 0.00
  fear: 0.17
  happy: 0.00
  sad: 0.12
  surprise: 0.00
  neutral: 99.52

🚫 Face not detected or error: invalid literal for int() with base 10: 'DESKTOP-UQ9OCE2'
DeepFace Results:
Dominant Emotion: sad
Face Confidence: 0
Region:
  x: 0
  y: 0
  w: 639
  h: 479
  left_eye: None
  right_eye: None
Emotions:
  angry: 0.00
  disgust: 0.00
  fear: 42.98
  happy: 0.00
  sad: 56.93
  surprise: 0.00
  neutral: 0.09

🚫 Face not detected or error: invalid literal for int() with base 10: 'DESKTOP-UQ9OCE2'
DeepFace Results:
Dominant Emotion: neutral
Face Confidence: 0.93
Region:
  x: 312
  y: 161
  w: 139
  h: 139
  left_eye: None
  right_eye: None
Emotions:
  angry: 0.02
  disgust: 0.00
  fear: 0.01
  happy: 0.00
  sad: 0.10
  surprise: 0.00
  neutral: 99.87

🚫 Face not detected or error: invalid literal for

KeyboardInterrupt: 